In [37]:
def remove_html(string):
    regex = re.compile(r'<[^>]+>')
    return regex.sub('{|}', string)

In [38]:
# search for indication of parental control tools
def verify_parental_control(self):
    words = ['parentalconsent', 'parental consent']
    result = find_words(self, words, True)
    return result

In [39]:
# search for age verification information in strings.xml
def verify_age(self):
    words = ['age verification']
    result = find_words(self, words, True)
    return result

In [40]:
# search for terms and conditions and privacy agreements
def verify_url(self):
    words = ['privacy', 'policy', 'terms', 'guidelines']
    url_found = find_url(self, words)
    return url_found

In [41]:
# check if geolocation data is defined in AndroidManifest.xml
def verify_geolocation(self):
    words = ['access fine location', 'access coarse location']
    result = find_words(self, words, True)
    return result

In [42]:
# combining dicts
def combine_dict(d1, d2):
    return {
        k: tuple(d[k] for d in (d1, d2) if k in d)
        for k in set(d1.keys()) | set(d2.keys())
    }

In [43]:
# remove special characters
def remove_special(self):
    characters = ['<', '"', '-', '_', ':', '=', '.', '/', '>', '[', ']', '  ', '  ']
    result = self
    for c in characters:
        result = result.replace(c, " ")
    return result

In [44]:
def find_rating(string):
    rating_keyword = '"contentRating":"'
    rating_pos = string.find(rating_keyword)
    rating_result = "Not Found"
    if rating_pos != -1:
        new_string = string[rating_pos + len(rating_keyword):]
        rating_pos = new_string.find('"')
        rating_result = new_string[0:rating_pos]
    return rating_result

In [45]:
# find package names in AndroidManifest.xml
def find_package(android_manifest):
    result = ""
    keyword = 'package="'
    found_index = android_manifest.find(keyword)
    if found_index != -1:
        ini_index = found_index + len(keyword)
        fin_index = ini_index + android_manifest[ini_index:].find('"')
        result = android_manifest[ini_index:fin_index]
    return result

In [46]:
# search for indication of report contents
def verify_content_report(self):
    words = ['block and report', 'block or report', 'report abuse', 'report as inappropriate', 'report bullying',
             'report comment', 'report content', 'report explicit image', 'report extremism', 'report hate speech',
             'report imminent danger', 'report inappropriate', 'report nsfw', 'report nudity', 'report or block',
             'report pornograph', 'report sexually explicit', 'report this contact', 'report this group',
             'report this member', 'report this photo', 'report this post', 'report this user', 'report this video',
             'report user', 'reportchatchild', 'reportchatpornography', 'reportchatviolence',
             'reporting harassment', 'reporting hateful', 'reporting nudity', 'reporting self harm',
             'reporting violence', 'thanks for reporting', 'the post you reported has been removed']
    result = find_words(self, words, True)
    return result

In [47]:
# find occurences of words in a corpus, with the option of searching for synonyms
def find_words(self, words, find_synonyms):
    found_words = []

    if find_synonyms:
        keywords = syn_comb(words)
    else:
        keywords = words

    for w1 in keywords:
        iteration = [m.start() for m in re.finditer(w1, self)]
        if len(iteration) != 0:
            found_word = w1 + ' ( ' + str(len(iteration)) + ' )'
            found_words.append(found_word)
    return ', '.join(found_words)

In [48]:
# find urls in a corpus of text
def find_url(string, words):
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex, string)
    relevant_url = list()
    for each_url in url:
        found = False
        for each_word in words:
            if each_url[0].find(each_word) != -1:
                found = True
        if found:
            relevant_url.append(each_url[0])
    relevant_url = [each_found_url.replace('\\', '') for each_found_url in relevant_url]
    relevant_url = list(dict.fromkeys(relevant_url))
    for each_found_url in relevant_url:
        try:
            content = str(requests.get(each_found_url, allow_redirects=True).content)
        except requests.exceptions.RequestException as e:
            content = ''
            
    return ' | '.join(relevant_url)

In [49]:
# combination of synonyms
def syn_comb(words):
    final_answer = list()
    for w in words:
        words_pos = pos_tag(word_tokenize(w))
        syns = {}
        for wp_word, wp_pos in words_pos:
            syn = [wp_word]
            for synset in wordnet.synsets(wp_word):
                for lemma in synset.lemmas():
                    if wp_pos == pos_tag(word_tokenize(lemma.name()))[0][1]:
                        syn.append(pos_tag(word_tokenize(lemma.name()))[0][0].replace('_', ' '))
            syn = list(dict.fromkeys(syn))
            syns[wp_word] = syn

        param_grid = ParameterGrid(syns)
        for dict_ in param_grid:
            combination = ''
            for key in syns:
                combination += dict_[key] + ' '
            combination = combination.strip()
            final_answer.append(combination)
    return final_answer

In [50]:
def find_data_safety(string):
    data_safety_keyword = '/store/apps/datasafety'
    data_safety_pos = string.find(data_safety_keyword)
    data_safety_result = ""
    new_string = ''
    if data_safety_pos != -1:
        new_string = string[data_safety_pos:]
        data_safety_pos = new_string.find('"')
        data_safety_result = "https://play.google.com" + new_string[0:data_safety_pos]
        new_string = requests.get(data_safety_result, allow_redirects=True).content.decode('utf-8')

    data_safety_pos_ini = new_string.find('>Data safety')
    data_safety_pos_fin = new_string.find('<div jscontroller="LCkxpb"')
    if data_safety_pos_ini != -1:
        new_string = remove_html(new_string[data_safety_pos_ini + 1:data_safety_pos_fin])

    ns_pos = new_string.find('{|}{|}')
    while ns_pos != -1:
        new_string = new_string.replace('{|}{|}', '{|}')
        ns_pos = new_string.find('{|}{|}')

    new_string = new_string.replace(' ', '')
    new_string = new_string.replace('expand_more{|}', '')
    new_string = new_string.replace('{|}info{|}', '{|}')
    new_string = new_string.replace('{|} &middot; ', ' - ')
    new_string = new_string.replace(' {|}Learn more{|} about how developers declare sharing.', '')
    new_string = new_string.replace('Learn more{|}', '')
    new_string = new_string.replace("Data safety{|}Here\\'s more information the developer has provided about the "
                                    'kinds of data this app may collect and share, and security practices the app may '
                                    'follow. Data practices may vary based on your app version, use, region, '
                                    'and age.\\xc2\\xa0{|}', '')

    if data_safety_result == '':
        new_string = 'No Data Safety Information Provided{|}' + new_string
    else:
        new_string = data_safety_result + '{|}' + new_string

    return new_string.split('{|}')

In [51]:
def ds_no_info(item):
    if item == 'No Data Safety Information Provided':
        return True
    else:
        return False

def ds_play_policy(item):
    if item == 'Committed to follow the Play Families Policy':
        return True
    else:
        return False

def ds_data_shared(item):
    if item == 'Data shared and for what purpose':
        return True
    else:
        return False

def ds_data_collected(item):
    if item == 'Data collected and for what purpose':
        return True
    else:
        return False

def ds_encrypted(item):
    if item == 'Data is encrypted in transit':
        return True
    else:
        return False

def ds_secure_conn(item):
    if item == 'Your data is transferred over a secure connection':
        return True
    else:
        return False

def ds_delete_data(item):
    if item == 'The developer provides a way for you to request that your data be deleted':
        return True
    else:
        return False
    
def ds_dot(col):
    ws.write(row, col, '•')

In [52]:
# main routine
import json
import os
import requests
import re

from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import ParameterGrid
from xlsxwriter import Workbook
from PIL import Image
import urllib.request

address = 'APK/'
dict_dirs = {}
dict_apps = {}

# creating a dict with the applications from the .json file
print('creating a dict with the applications from the .json file')
f = open('TEXT_FILES/list_of_applications.json', 'r')
data = json.load(f)

for groups in data['organic_results']:
    for packages in groups['items']:
        ttl = packages['title']
        lnk = packages['link']
        prd = packages['product_id']
        api = packages['serpapi_link']
        thu = packages['thumbnail']
        if 'rating' in packages:
            rat = packages['rating']
        else:
            rat = 'x.x'
        
        key = prd
        value = [ttl,lnk,api,thu,rat]
        
        dict_apps[key] = value

# verifying contents of downloaded APKs
print('verifying contents of downloaded APKs')
with os.scandir(address) as entries:
    counter = 0
    for entry in entries:
        counter = counter + 1
        directory = address + '/' + entry.name
        am_file = directory + '/' + 'AndroidManifest.xml'
        st_file = directory + '/' + 'strings.xml'
        if os.path.isdir(directory):
            pac = ''
            geo = ''
            url = ''
            age = ''
            par = ''
            rep = ''
            if os.path.isfile(am_file):
                with open(am_file, encoding="utf8") as f:
                    con = f.read()
                    pac = find_package(con)
                    con = remove_special(con).lower()
                    geo = verify_geolocation(con)
                    f.close()
            if os.path.isfile(st_file):
                with open(st_file, encoding="utf8") as f:
                    con = f.read()
                    url = verify_url(con)
                    con = remove_special(con).lower()
                    age = verify_age(con)
                    par = verify_parental_control(con)
                    rep = verify_content_report(con)
                    f.close()
            key = pac
            value = [geo, age, par, rep, url]
            dict_dirs[key] = value

# merging both dicts
print('merging both dicts')
dict_new = combine_dict(dict_apps, dict_dirs)
        
# creating the Excel file
print('creating the Excel file')
wb = Workbook('TEXT_FILES/ChildDataVerif_APPS.xlsx')
ws = wb.add_worksheet()
ws.set_comments_author('Automatically Generated')
ws.set_default_row(30)
ws.set_column(0, 0,  5)
ws.set_column(1, 2, 25)
row = 0
ws.write(row,  1, 'App Name')
ws.write(row,  2, 'Product ID')
ws.write(row,  3, 'Rating')
ws.write(row,  4, 'Google Store Link')
ws.write(row,  5, 'Age Appropriate')
ws.write(row,  6, 'Teacher Approved')
ws.write(row,  7, 'Geolocation')
ws.write(row,  8, 'Age Verification')
ws.write(row,  9, 'Parental Controls')
ws.write(row, 10, 'Reporting')
ws.write(row, 11, 'No Data Safety Information Provided')
ws.write(row, 12, 'Committed to follow the Play Families Policy')
ws.write(row, 13, 'Data shared and for what purpose')
ws.write(row, 14, 'Data collected and for what purpose')
ws.write(row, 15, 'Data is encrypted in transit')
ws.write(row, 16, 'Your data is transferred over a secure connection')
ws.write(row, 17, 'The developer provides a way for you to request that your data be deleted')
ws.write(row, 18, 'URLs')
ws.write(row, 19, 'Data Safety')

for entry in dict_new:
    row += 1
    
    ws.write(row,  1, dict_new[entry][0][0])
    ws.write(row,  2, entry)
    ws.write(row,  3, dict_new[entry][0][4])
    ws.write(row,  4, dict_new[entry][0][1])
    if dict_new[entry][1][0] != '':
        ds_dot(7)
        ws.write_comment(row,  7, dict_new[entry][1][0])
    if dict_new[entry][1][1] != '':
        ds_dot(8)
        ws.write_comment(row,  8, dict_new[entry][1][1])
    if dict_new[entry][1][2] != '':
        ds_dot(9)
        ws.write_comment(row,  9, dict_new[entry][1][2])
    if dict_new[entry][1][3] != '':
        ds_dot(10)
        ws.write_comment(row, 10, dict_new[entry][1][3])
        
    ws.write(row, 18, dict_new[entry][1][4])
    
    # image retrieval, manipulation and insertion
    image = 'TEXT_FILES/img/' + entry.replace('.', '_') + '.jpg'
    urllib.request.urlretrieve(dict_new[entry][0][3], image)
    thumb = Image.open(image)
    if len(thumb.split()) == 3:
        red, green, blue = thumb.resize((30, 30)).split()
    else:
        if len(thumb.split()) == 4:
            red, green, blue, alpha = thumb.resize((30, 30)).split()
    thumbnail = Image.merge('RGB', (red, green, blue))
    thumbnail.save(image)
    image = 'TEXT_FILES/img/' + entry.replace('.', '_') + '.jpg'
    ws.insert_image(row, 0, image)
    
    # Age Appropriate, Teacher Approved and Data Safety information
    try:
        content = str(requests.get(dict_new[entry][0][1], allow_redirects=True).content)
        ws.write(row, 5, find_rating(content))
        if content.find("Teacher Approved") != -1:
            ws.write(row, 6, "Yes")
        else:
            ws.write(row, 6, "No")

        ss_content = find_data_safety(content)
        ss_ini_col = 19
        ss_counter = 0
        for ds_item in ss_content:
            ws.write(row, ss_ini_col + ss_counter, ds_item)
            if ds_no_info(ds_item):
                ds_dot(11)
            if ds_play_policy(ds_item):
                ds_dot(12)
            if ds_data_shared(ds_item):
                ds_dot(13)
            if ds_data_collected(ds_item):
                ds_dot(14)
            if ds_encrypted(ds_item):
                ds_dot(15)
            if ds_secure_conn(ds_item):
                ds_dot(16)
            if ds_delete_data(ds_item):
                ds_dot(17)
            ss_counter = ss_counter + 1

    except requests.exceptions.RequestException as e:
        ws.write(row, 0, "File Not Found")
    
wb.close()
print('Done!')

creating a dict with the applications from the .json file
verifying contents of downloaded APKs
merging both dicts
creating the Excel file
Done!


In [7]:
import json
import requests

# creating a dict with the applications from the .json file
print('searching SerpApi weblink information')
f = open('TEXT_FILES/list_of_applications.json', 'r')
data = json.load(f)

for groups in data['organic_results']:
    for packages in groups['items']:
        api = packages['serpapi_link']
        #content = str(requests.get(api, allow_redirects=True).content)
        print(api)

searching SerpApi weblink information
https://serpapi.com/search.json?engine=google_play_product&gl=uk&hl=en&product_id=com.google.android.apps.youtube.kids&store=apps
https://serpapi.com/search.json?engine=google_play_product&gl=uk&hl=en&product_id=com.tocaboca.tocahairsalon4&store=apps
https://serpapi.com/search.json?engine=google_play_product&gl=uk&hl=en&product_id=com.lego.common.legolife&store=apps
https://serpapi.com/search.json?engine=google_play_product&gl=uk&hl=en&product_id=com.playdatedigital.tonka&store=apps
https://serpapi.com/search.json?engine=google_play_product&gl=uk&hl=en&product_id=com.rosytales.candyhousecleaning&store=apps
https://serpapi.com/search.json?engine=google_play_product&gl=uk&hl=en&product_id=com.tigerqueen.kids_doodle_paint&store=apps
https://serpapi.com/search.json?engine=google_play_product&gl=uk&hl=en&product_id=com.genius.girls.games.Kids.Car.Wash.Auto.Garage.Workshop&store=apps
https://serpapi.com/search.json?engine=google_play_product&gl=uk&hl=en&

In [23]:
import os
import json
address = 'TEXT_FILES/SerpApi/'
with os.scandir(address) as entries:
    for entry in entries:
        print(entry)
        f = open(entry, 'r')
        data = json.load(f)
        save_data = []
        save_data.append(data['product_info']['title'])
        save_data.append(data['product_info']['authors'][0]['name'])
        rat = 'x.x'
        if 'rating' in data['product_info']:
            rat = str(data['product_info']['rating'])
        save_data.append(rat)
        rev = ''
        if 'reviews' in data['product_info']:
            rev = str(data['product_info']['reviews'])
        save_data.append(rev)
        save_data.append(data['product_info']['content_rating']['text'])
        save_data.append(data['product_info']['downloads'])
        save_data.append(data['categories'][0]['name'])
        for key in data['developer_contact']:
            save_data.append(data['developer_contact'][key])

        f = open('TEXT_FILES/SerpApi.txt', 'a')
        f.write(", ".join(save_data))
        f.write('\n')
        f.close()

<DirEntry 'com.askfm.json'>
<DirEntry 'com.ravensburger.puzzle.junior.json'>
<DirEntry 'com.rosytales.candyhousecleaning'>
<DirEntry 'com.rvappstudios.math.kids.counting.json'>
<DirEntry '1_com.instagram.android.json'>
<DirEntry 'com.lescapadou.tracingfree.json'>
<DirEntry 'com.starfall.StarfallABCs.json'>
<DirEntry 'com.pokemon.pokemonplayhouse.json'>
<DirEntry 'com.bimiboo.mysteries.json'>
<DirEntry 'com.RespectStudios.WheresSamantha.json'>
<DirEntry 'com.rickyzoom.welcometowheelford.json'>
<DirEntry 'ru.ok.android.json'>
<DirEntry '1_com.duolingo.literacy.json'>
<DirEntry 'com.pinterest.json'>
<DirEntry 'com.tocaboca.tocalifeworld'>
<DirEntry 'br.com.babyanimalsgame.json'>
<DirEntry '1_sh.whisper.json'>
<DirEntry 'com.storytoys.disney.pixar.coloring.princess.googleplay.json'>
<DirEntry 'com.vsco.cam.json'>
<DirEntry 'com.google.android.youtube.json'>
<DirEntry 'com.sunny.halloween.coloring.book.json'>
<DirEntry '1_com.ideafun.paint.book.json'>
<DirEntry 'com.sagosago.School.googlepl